In [ ]:
%pwd
%cd ../..

In [ ]:
from functools import partial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from deeplifting.problems import PROBLEMS_BY_NAME
from deeplifting.optimization import (
    run_deeplifting,
    run_differential_evolution,
    run_dual_annealing,
    run_ipopt,
    run_pygranso,
)
from deeplifting.utils import create_optimization_plot

In [ ]:
problem_name = 'ex8_1_6'
problem = PROBLEMS_BY_NAME[problem_name]

In [ ]:
# # Run pygranso
# outputs = run_pygranso(problem, trials=1)

In [ ]:
outputs = run_deeplifting(problem, trials=10)

In [ ]:
%debug

In [ ]:
results = outputs['final_results']

In [ ]:
results

In [ ]:
fig = create_optimization_plot(problem_name, problem, results, colormap='autumn_r')

In [ ]:
fig.savefig(
    '/home/rydevera3/data-science/umn_research/Deeplifting'
    f'/images/{problem_name}_surface_and_contour-autumn.png'
)

In [ ]:
import numpy as np

results = np.zeros((1, 1, 3))
trial = 0
x = np.array([5.9999, 5.9999])

In [ ]:
objective = problem['objective']

In [ ]:
objective(x, results, trial)